In [43]:
# ! pip install pymongo spicy
# ! pip install matplotlib seaborn pandas numpy
# ! pip install pingouin
! pip install spicy krippendorff 


[notice] A new release of pip is available: 23.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [44]:
%matplotlib inline
import pandas as pd
import numpy as np
import math
import seaborn as sns
import matplotlib.pyplot as plt
import bson, json
from datetime import datetime
from collections import defaultdict
from pingouin import reliability
from scipy.stats import shapiro, f_oneway, ttest_ind
import krippendorff as kd


# Descriptive Statisctic

## Method definition

In [2]:
# Convert bson file to dataframe
def bson_to_dataframe(filename):
    with open(f'./bson/{filename}.bson', 'rb') as f:
        data = bson.decode_all(f.read())
        df = pd.DataFrame(data)
    return df

In [3]:
# Convert json file to dataframe
def json_to_dataframe(filename):
    with open(f'./json/{filename}.json', 'rb') as f:
        data = json.load(f)
        df = pd.DataFrame(data)
    return df

In [4]:
# Calculate duration 
'''
Title: Calculate Time Difference Between Two Pandas Columns in Hours and Minutes
Author: cottontail
Date: Feb 16, 2023
Availability: https://stackoverflow.com/questions/22923775/calculate-time-difference-between-two-pandas-columns-in-hours-and-minutes
'''
def cal_total_time(start_time, end_time):
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)
    duration = end_time - start_time
    duration_readable = "{:02}:{:02}:{:02}".format(int(duration.seconds / 3600), int((duration.seconds % 3600) / 60), int(duration.seconds % 60))

    return duration_readable

In [5]:
# Extract results of surveys
def extract_survey_results(dataframe):
    results = dataframe['results']

    '''
    Title: Convert Pandas Series to Dict in Python (with code)
    Author: Riddhima Agarwal
    Date: Jun 10, 2023
    Availability: https://favtutor.com/blogs/pandas-series-to-dict#:~:text=A%20Pandas%20Series%20can%20be,the%20Pandas%20Series%20as%20values
    '''
    results_list = results.to_list()
    results_df = pd.DataFrame(results_list)

    dataframe_joined = dataframe.join(results_df)
    columns = ['_id', 'results', 'createdAt', 'updatedAt', '__v']
    dataframe = dataframe_joined.drop(columns=columns, axis=1)

    return dataframe

In [120]:
'''
Title: Inter-rater reliability calculation for multi-raters data
Author: DieseRobin
Date: Nov 25, 2021
Availability: https://stackoverflow.com/questions/56481245/inter-rater-reliability-calculation-for-multi-raters-data
'''
def cal_kippendorf_alpha(df, col):
    matrix = df.pivot(index='user_id', columns='task_id', values=col).values.astype(int)
    alpha = kd.alpha(matrix, level_of_measurement='nominal')
    return alpha

In [ ]:
# Draw heatmap to show level of familiarity, complexity and satisfaction for each user and task
def draw_heatmap(df, selected_column, cmap):
    heatmap_data = df.pivot_table(index='user_id', columns='task_id', values=selected_column, fill_value=0)

    plt.figure(figsize=(6, 3))
    sns.heatmap(heatmap_data, annot=True, fmt=".1f", cmap=cmap, linewidths=.5)
    plt.title(f'{selected_column} heatmap')
    plt.xlabel('Task ID')
    plt.ylabel('User ID')
    plt.savefig(f'plot_output/{selected_column}_heatmap.jpg')

## Data Exploration

In [123]:
users_df = bson_to_dataframe("users")
prestudies_df = bson_to_dataframe("prestudies")
pretasks_df = bson_to_dataframe("pretasks")
sessions_df = bson_to_dataframe("sessions")
searchinteractions_df = bson_to_dataframe("searchinteractions")
documents_df = bson_to_dataframe("documents")
chatinteractions_df = bson_to_dataframe("chatinteractions")
posttasks_df = bson_to_dataframe("posttasks")
poststudies_df = bson_to_dataframe("poststudies")

tasks_df = json_to_dataframe("tasks")

In [97]:
users_df['study_total_time'] = users_df.apply(lambda row: cal_total_time(row['study_start_time'], row['study_end_time']), axis=1)
users_df = users_df[['user_id', 'study_total_time']]
users_df

,user_id,study_total_time
0,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,00:11:21
1,e0711a26-c616-463c-9b05-f0afaf871f31,01:23:42
2,15c09788-9078-4e28-9ec0-91d634c5b9ca,00:30:51
3,34f7c191-ff70-407c-b818-49e35cbb068f,00:15:01
4,2fbf76df-1101-4ac6-b75b-f49df2266232,00:32:11
5,fe53df51-c33e-4114-986c-c9efcca99259,00:38:17
6,525d6692-2c89-4871-b945-28f5d35a789b,00:10:58
7,d235cab9-2a67-4667-8857-a51566cffaf7,00:18:22
8,7c2b4159-df97-40a1-8376-23df2a4711ba,00:35:29
9,d80ef3e3-ebb7-40a5-9ce1-812e754809aa,00:51:39


Clean data to avoid bias

In [98]:
# filter only data of people who did the entire study
def filter_users(df):
    df = df[df['user_id'].isin(users_df['user_id'])]
    return df

In [99]:
prestudies_df = filter_users(prestudies_df)
pretasks_df = filter_users(pretasks_df)
sessions_df = filter_users(sessions_df)
searchinteractions_df = filter_users(searchinteractions_df)
chatinteractions_df = filter_users(chatinteractions_df)
posttasks_df = filter_users(posttasks_df)
poststudies_df = filter_users(poststudies_df)

documents_df = documents_df[documents_df['interaction_id'].isin(searchinteractions_df['interaction_id'])]

### Data Analysis

### a. Survey Data

#### 1. System experience

In [124]:
prestudies_df = extract_survey_results(prestudies_df)
prestudies_df

,user_id,search-engine-usage,Ai-copilots-usage,onl-search-freq,ai-search-freq
0,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,6,4,Once per day,Once per week
1,e0711a26-c616-463c-9b05-f0afaf871f31,7,5,Multiple times per day,Multiple times per week
2,15c09788-9078-4e28-9ec0-91d634c5b9ca,7,2,Multiple times per day,Once per week
3,34f7c191-ff70-407c-b818-49e35cbb068f,7,6,Multiple times per day,Once per day
4,2fbf76df-1101-4ac6-b75b-f49df2266232,7,7,Multiple times per day,Multiple times per day
5,fe53df51-c33e-4114-986c-c9efcca99259,7,7,Multiple times per day,Multiple times per day
6,525d6692-2c89-4871-b945-28f5d35a789b,7,6,Multiple times per day,Once per day
7,d235cab9-2a67-4667-8857-a51566cffaf7,7,1,Multiple times per day,Never
8,7c2b4159-df97-40a1-8376-23df2a4711ba,6,6,Multiple times per day,Multiple times per week
9,d80ef3e3-ebb7-40a5-9ce1-812e754809aa,7,5,Multiple times per week,Multiple times per week


In [125]:
prestudies_alpha = reliability.cronbach_alpha(data=prestudies_df[['search-engine-usage', 'Ai-copilots-usage']])
prestudies_alpha

(-0.06504065040650353, array([-2.959,  0.713]))

In [127]:
df = prestudies_df[['user_id', 'onl-search-freq', 'ai-search-freq']]
df

,user_id,onl-search-freq,ai-search-freq
0,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,Once per day,Once per week
1,e0711a26-c616-463c-9b05-f0afaf871f31,Multiple times per day,Multiple times per week
2,15c09788-9078-4e28-9ec0-91d634c5b9ca,Multiple times per day,Once per week
3,34f7c191-ff70-407c-b818-49e35cbb068f,Multiple times per day,Once per day
4,2fbf76df-1101-4ac6-b75b-f49df2266232,Multiple times per day,Multiple times per day
5,fe53df51-c33e-4114-986c-c9efcca99259,Multiple times per day,Multiple times per day
6,525d6692-2c89-4871-b945-28f5d35a789b,Multiple times per day,Once per day
7,d235cab9-2a67-4667-8857-a51566cffaf7,Multiple times per day,Never
8,7c2b4159-df97-40a1-8376-23df2a4711ba,Multiple times per day,Multiple times per week
9,d80ef3e3-ebb7-40a5-9ce1-812e754809aa,Multiple times per week,Multiple times per week


In [131]:
SE_freq = df.groupby(['onl-search-freq']).size().reset_index(name='count')
SE_freq 

,onl-search-freq,count
0,Multiple times per day,9
1,Multiple times per week,1
2,Once per day,1


In [132]:
AI_freq = df.groupby(['ai-search-freq']).size().reset_index(name='count')
AI_freq 

,ai-search-freq,count
0,Multiple times per day,2
1,Multiple times per week,3
2,Never,2
3,Once per day,2
4,Once per week,2


#### 2. Pre-Task Data Analysis

In [100]:
pretasks_df = extract_survey_results(pretasks_df)
pretasks_df.head(10)


,user_id,task_id,familiarity,complexity
0,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task1,3,3
1,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task2,1,5
2,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task4,3,6
3,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task3,4,5
4,e0711a26-c616-463c-9b05-f0afaf871f31,task2,4,2
5,e0711a26-c616-463c-9b05-f0afaf871f31,task3,5,4
6,e0711a26-c616-463c-9b05-f0afaf871f31,task1,3,3
7,e0711a26-c616-463c-9b05-f0afaf871f31,task4,2,5
8,15c09788-9078-4e28-9ec0-91d634c5b9ca,task3,7,2
9,15c09788-9078-4e28-9ec0-91d634c5b9ca,task4,7,2


2.1 Familiarity

In [103]:
familiarity_alpha = cal_kippendorf_alpha(pretasks_df, 'familiarity')
familiarity_alpha

0.08405612244897964

2.2 Pre complexity ratings

In [105]:
pre_complexity_alpha = cal_kippendorf_alpha(pretasks_df, 'complexity')
pre_complexity_alpha

-0.006382978723404209

2.3 Visualization

In [107]:
# familiarity_heatmap = draw_heatmap(df=pretasks_df, selected_column='familiarity', cmap='YlGnBu')

In [108]:
# pre_complexity_heatmap = draw_heatmap(df=pretasks_df, selected_column='complexity', cmap='BuPu')

#### 3. Post-Task Data Analysis

In [109]:
posttasks_df = extract_survey_results(posttasks_df)
posttasks_df.head(10)

,user_id,task_id,answer,complexity-rate,result-system,satisfaction-rate,attention
0,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task1,Unhealthy Lifestyle e.g. Smoking and excessive...,3,Generative AI (ChatGPT),6,Okay
1,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task2,"Solenopsis invicta, also known as the red impo...",2,Generative AI (ChatGPT),7,Okay
2,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task4,1. Cost: Conventional methods of home heating ...,3,Generative AI (ChatGPT),5,Okay
3,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task3,Some food additives that may pose a risk to ph...,3,Generative AI (ChatGPT),7,Okay
4,e0711a26-c616-463c-9b05-f0afaf871f31,task2,Solenopsis invicta,2,Both Google and ChatGPT,6,Okay
5,e0711a26-c616-463c-9b05-f0afaf871f31,task3,1. Artificial sweeteners\n2. Sodium nitrate an...,5,Generative AI (ChatGPT),5,Okay
6,e0711a26-c616-463c-9b05-f0afaf871f31,task1,"Many factors can affect blood pressure, includ...",4,Normal search engine (Google),5,Okay
7,e0711a26-c616-463c-9b05-f0afaf871f31,task4,1. Cost\n2. Energy Efficiency\n3. Environmenta...,5,Both Google and ChatGPT,5,Okay
8,15c09788-9078-4e28-9ec0-91d634c5b9ca,task3,Every food combines of many nutrients. It is t...,2,Normal search engine (Google),4,Okay
9,15c09788-9078-4e28-9ec0-91d634c5b9ca,task4,It depends on your demand. Fossil are cheaper ...,3,Generative AI (ChatGPT),5,Okay


3.1 Task complexity ratings

In [110]:
post_complexity_alpha = cal_kippendorf_alpha(posttasks_df, 'complexity-rate')
post_complexity_alpha

-0.023291139240506187

3.2 Satisfaction ratings

In [111]:
satisfaction_alpha = cal_kippendorf_alpha(posttasks_df, 'satisfaction-rate')
satisfaction_alpha

-0.004558404558404172

3.3 Visualization

In [112]:
# post_complexity_heatmap = draw_heatmap(df=posttasks_df, selected_column='complexity-rate', cmap='Blues')

In [113]:
# satisfaction_heatmap = draw_heatmap(df=posttasks_df, selected_column='satisfaction-rate', cmap='Greens')

##### 3.3 Answers for evaluation

In [116]:
answers_data = posttasks_df[['user_id', 'task_id', 'answer']]
answers_data.head(10)

,user_id,task_id,answer
0,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task1,Unhealthy Lifestyle e.g. Smoking and excessive...
1,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task2,"Solenopsis invicta, also known as the red impo..."
2,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task4,1. Cost: Conventional methods of home heating ...
3,4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a,task3,Some food additives that may pose a risk to ph...
4,e0711a26-c616-463c-9b05-f0afaf871f31,task2,Solenopsis invicta
5,e0711a26-c616-463c-9b05-f0afaf871f31,task3,1. Artificial sweeteners\n2. Sodium nitrate an...
6,e0711a26-c616-463c-9b05-f0afaf871f31,task1,"Many factors can affect blood pressure, includ..."
7,e0711a26-c616-463c-9b05-f0afaf871f31,task4,1. Cost\n2. Energy Efficiency\n3. Environmenta...
8,15c09788-9078-4e28-9ec0-91d634c5b9ca,task3,Every food combines of many nutrients. It is t...
9,15c09788-9078-4e28-9ec0-91d634c5b9ca,task4,It depends on your demand. Fossil are cheaper ...


In [119]:
# get the answer for task 3 of user 1 
filtered_data = answers_data[(answers_data['user_id'] == '4edf0ff5-b5aa-4957-9431-4fbf4cacaf6a') & (answers_data['task_id'] == 'task3')]
answer = filtered_data['answer'].values[0] if not filtered_data.empty else None
answer

'Some food additives that may pose a risk to physical health include:\n\n1. Artificial colors: Some artificial colors, such as Red 40 and Yellow 6, have been linked to behavioral issues and hyperactivity in children.\n\n2. Artificial sweeteners: Sweeteners like aspartame and saccharin have been associated with negative health effects, including headaches, digestive issues, and potential links to cancer.\n\n3. Preservatives: Preservatives like sodium nitrate and sulfites have been linked to allergies, asthma, and other adverse reactions in some individuals.\n\n4. Trans fats: Hydrogenated oils, which contain trans fats, have been linked to an increased risk of heart disease and other chronic health conditions.\n\n5. MSG (monosodium glutamate): This flavor enhancer has been linked to headaches, sweating, and other symptoms in some individuals.\n\nThese additives are likely to be listed on grocery store labels under their specific names or as part of a broader category (e.g. "artificial co

#### 4 Demographic Data

In [ ]:
poststudies_df = extract_survey_results(poststudies_df)
poststudies_df

In [ ]:
# filter only participants who need VP-Stunden
VP_data = poststudies_df[(poststudies_df['Vp-stunden'] == True)]
matrikel_nr_list = VP_data['matrikel-nr']
matrikel_nr_list

'''
Title: Writing a list to a file with Python, with newlines
Author: Alex Martelli
Date: May 22, 2009
Availability: https://stackoverflow.com/questions/899103/writing-a-list-to-a-file-with-python-with-newlines
'''
with open('matrikel_nr_list.txt', 'w') as f:
    for line in matrikel_nr_list:
        f.write(f"{line}\n")

### b. Interaction Data 

In [ ]:
tasks_df.rename(columns={'id':'task_id'}, inplace=True)
tasks_df

In [ ]:
sessions_df['session_total_time'] = sessions_df.apply(lambda row: cal_total_time(row['session_start_time'], row['session_end_time']), axis=1)
columns = ['_id', '__v', 'session_start_time', 'session_end_time']
sessions_df = sessions_df.drop(columns=columns, axis=1)

In [ ]:
sessions_df

##### 1. Time overall

In [ ]:
sessions_merged_df= pd.merge(sessions_df, tasks_df, on='task_id', how='left')
sessions_merged_df = sessions_merged_df.drop(['title', 'desc'], axis=1)

In [ ]:
sessions_merged_df 

In [ ]:
task_time_df = sessions_merged_df[['task_id', 'session_total_time']]
task_time_df

In [ ]:
# calculate the overall time for each task
def cal_time_each_task(dataframe):
    task_time_list = dataframe.to_dict('records')

    task_time_dict = {}

    for entry in task_time_list:
        task_id = entry["task_id"]
        session_time = datetime.strptime(entry["session_total_time"], "%H:%M:%S")
        if task_id in task_time_dict:
            task_time_dict[task_id] += session_time - datetime(1900, 1, 1)
        else:
            task_time_dict[task_id] = session_time - datetime(1900, 1, 1)

    return task_time_dict

In [ ]:
# Visualization using horizontal bar chart
'''
Title: Bar Plot in Matplotlib
Author: jeeteshgavand
Date: 04 Mar, 2021
Availability: https://www.geeksforgeeks.org/bar-plot-in-matplotlib/
'''
def visualize_time_each_task(task_time_dict):
    # Figure Size
    fig, ax = plt.subplots(figsize =(8, 3))
    
    task_overalltime_minutes = {key: value.total_seconds()/60 for key, value in task_time_dict.items()}

    tasks = list(task_overalltime_minutes.keys())
    minutes = list(task_overalltime_minutes.values())
    # Horizontal Bar Plot
    ax.barh(tasks, minutes)
    
    # Remove axes splines
    for s in ['top', 'bottom', 'left', 'right']:
        ax.spines[s].set_visible(False)
    
    # Remove x, y Ticks
    ax.xaxis.set_ticks_position('none')
    ax.yaxis.set_ticks_position('none')
    
    # Add padding between axes and labels
    ax.xaxis.set_tick_params(pad = 5)
    ax.yaxis.set_tick_params(pad = 10)
    
    # Add x, y gridlines
    ax.grid(b = True, color ='grey',
            linestyle ='-.', linewidth = 0.5,
            alpha = 0.2)
    
    # Show top values 
    ax.invert_yaxis()
    
    # Add annotation to bars
    for i in ax.patches:
        plt.text(i.get_width()+0.2, i.get_y()+0.5, 
                str(round((i.get_width()), 2)),
                fontsize = 10, fontweight ='bold',
                color ='grey')
    
        # Adding labels and title
    ax.set_xlabel('Minutes')
    ax.set_title('Overall Time for each Task',
                loc ='left', )
    ax.figure.savefig("plot_output/Overall_Time_each_Task.jpg")

In [ ]:
# Visualization using vertical bar chart
def visualize_overalltime(task_time_dict): 
    simple_tasks_overalltime = (task_time_dict['task1'] + task_time_dict['task2']).total_seconds()/60
    complex_tasks_overalltime =(task_time_dict['task3'] + task_time_dict['task4']).total_seconds()/60

    # Data for plotting
    tasks = ['Simple Tasks', 'Complex Tasks']
    times = [simple_tasks_overalltime, complex_tasks_overalltime]

    fig = plt.figure(figsize = (5, 3))

    # Plotting the bar chart
    plt.bar(tasks, times, color ='maroon')

    # Adding labels and title
    plt.xlabel('Task Type')
    plt.ylabel('Overall Time (minutes)')
    plt.title('Overall Time for Simple vs Complex Tasks')
    plt.savefig("plot_output/overalltime.jpg")

In [ ]:
# calculate the overall time for each type of tasks
def print_overalltime(task_time_dict):
    
    # Convert total times to string format
    simple_tasks_overalltime = str(task_time_dict['task1'] + task_time_dict['task2'])
    complex_tasks_overalltime = str(task_time_dict['task3'] + task_time_dict['task4'])

    for task_id, total_time in task_time_dict.items():
        task_time_dict[task_id] = str(total_time)

    # print overalltime
    print("Overall time for each task:")
    for task_id, total_time in task_time_dict.items():
        print(f"{task_id}: {total_time}")

    print('Overall time for simple tasks: ' + simple_tasks_overalltime)
    print('Overall time for complex tasks: ' + complex_tasks_overalltime)

In [ ]:
task_overalltime = cal_time_each_task(task_time_df)
visualize_time_each_task(task_overalltime)
visualize_overalltime(task_overalltime)
print_overalltime(task_overalltime)

##### 2. Queries

2.1. Search Queries

In [ ]:
# Count number of queries each task when interacting with SE or Chat
def count_queries_each_task(dict):
    queries_counts = defaultdict(int)

    for item in dict:
        queries_counts[item['task_id']] += 1

    return queries_counts

In [ ]:
task_session_df = sessions_merged_df[['session_id', 'task_id']]
task_session_df 

In [ ]:
cols = ['_id', '__v', 'createdAt', 'updatedAt']
searchinteractions_df = searchinteractions_df.drop(columns=cols, axis=1)
documents_df = documents_df.drop(columns=cols, axis=1)

In [ ]:
googlesearch_merged_df = pd.merge(searchinteractions_df, task_session_df, on='session_id', how='inner')
googlesearch_merged_df.head(10)

In [ ]:
task_queries_SE_df = googlesearch_merged_df[['query', 'task_id']]
task_queries_SE_df

In [ ]:
task_queries_SE_dict = task_queries_SE_df.to_dict('records')
task_queries_SE_dict

2.2. Chat Queries

In [ ]:
chatinteractions_df = chatinteractions_df[['user_id', 'session_id', 'interaction_type', 'messages']]
chatinteractions_df

In [ ]:
chat_merged_df = pd.merge(chatinteractions_df, task_session_df, on='session_id', how='inner')
chat_merged_df.head(10)

In [ ]:
def extract_chat_queries():  
    task_query_AI_dict = chat_merged_df[['messages', 'task_id']].to_dict() 
    extracted_data = []
    for key, value in task_query_AI_dict['messages'].items():
        if value is not None:
            for task_id, sublist in enumerate(value):
                for message in sublist:
                    if not message['isBot']:
                        extracted_data.append({'task_id': task_query_AI_dict['task_id'][key], 'query': message['text']})
    return extracted_data

In [ ]:
task_queries_AI_dict = extract_chat_queries()
task_queries_AI_dict

In [ ]:
task_queries_AI_df = pd.DataFrame(task_queries_AI_dict)
task_queries_AI_df 

2.3. Number of queries each system

In [ ]:
queries_SE_counts = count_queries_each_task(task_queries_SE_dict)
queries_SE_counts_df = pd.DataFrame(list(queries_SE_counts .items()), columns=['Task ID', 'Num Queries SE'])
queries_SE_counts_df

In [ ]:
queries_AI_counts = count_queries_each_task(task_queries_AI_dict)
queries_AI_counts_df = pd.DataFrame(list(queries_AI_counts .items()), columns=['Task ID', 'Num Queries AI'])
queries_AI_counts_df

In [ ]:
queries_system_df = pd.merge(queries_AI_counts_df, queries_SE_counts_df, on='Task ID', how='outer')
queries_system_df

2.4. Mean query length each system

In [ ]:
def cal_mean_query_length (dict, system):
    # Dictionary to store total query length and count for each task
    task_lengths = defaultdict(lambda: {'total_length': 0, 'count': 0})

    # Calculate total query length and count for each task
    for query in dict:
        task_id = query['task_id']
        query_length = len(query['query'].split())  # Split query by space and count words
        task_lengths[task_id]['total_length'] += query_length
        task_lengths[task_id]['count'] += 1

    # Calculate mean query length for each task
    mean_lengths = {task_id: task_info['total_length'] / task_info['count'] for task_id, task_info in task_lengths.items()}

    df = pd.DataFrame(mean_lengths.items(), columns=['Task ID', f"Mean Length Query{system}"])
    return df

In [ ]:
mean_length_queryAI_df = cal_mean_query_length(task_queries_AI_dict, "AI")
mean_length_queryAI_df

In [ ]:
mean_length_querySE_df = cal_mean_query_length(task_queries_SE_dict, "SE")
mean_length_querySE_df

In [ ]:
mean_query_length_df = pd.merge(mean_length_queryAI_df, mean_length_querySE_df, on='Task ID', how='outer')
mean_query_length_df

* Visualization 

In [ ]:
# Visualization using multiple bar charts
'''
Title: Bar Plot in Matplotlib
Author: jeeteshgavand
Date: 04 Mar, 2021
Availability: https://www.geeksforgeeks.org/bar-plot-in-matplotlib/
'''
def visualize_bar_chart(df, col1, col2, ylabel): 
    barWidth = 0.25
    fig = plt.subplots(figsize =(6, 3)) 

    list1 = df[col1]
    list2 = df[col2]
    
    br1 = np.arange(len(list1)) 
    br2 = [x + barWidth for x in br1] 
    
    plt.bar(br1, list1, color ='r', width = barWidth, 
            edgecolor ='grey', label =col1) 
    plt.bar(br2, list2, color ='g', width = barWidth, 
            edgecolor ='grey', label =col2) 

    plt.xlabel('Task ID', fontsize = 15) 
    plt.ylabel( ylabel, fontsize = 15) 
    plt.xticks([r + barWidth for r in range(len(list1))], df['Task ID'])
    
    plt.legend()
    plt.savefig(f"plot_output/{ylabel}.jpg")

In [ ]:
num_Queries_AI ='Num Queries AI'
num_Queries_SE ='Num Queries SE'
visualize_bar_chart(queries_system_df, num_Queries_AI, num_Queries_SE, "Number of Queries")

In [ ]:
mean_length_AI = "Mean Length QueryAI"
mean_length_SE = "Mean Length QuerySE"
visualize_bar_chart(mean_query_length_df, mean_length_AI, mean_length_SE, "Mean Query Length")

In [ ]:
# Visualize using boxplots
def visualize_boxplot(df, system):
    df['query_length'] = df['query'].apply(len)

    plt.figure(figsize=(6,3))
    df.boxplot(column='query_length', by='task_id')
    plt.title(f'Boxplot of Query Length {system} for Each Task')
    plt.xlabel('Task ID')
    plt.ylabel('Query Length')
    plt.suptitle('')
    plt.savefig(f"plot_output/query_length_{system}.jpg")

In [ ]:
visualize_boxplot(task_queries_AI_df, "AI")

In [ ]:
visualize_boxplot(task_queries_SE_df, "SE")

##### 3. Doc position analysis

In [ ]:
googlesearch_docs_df = pd.merge(googlesearch_merged_df, documents_df, on='interaction_id', how='inner')
googlesearch_docs_df.head(10)

In [ ]:
googlesearch_docs_df = googlesearch_docs_df[['task_id', 'doc_position', 'doc_page_viewed']]
googlesearch_docs_df.head(10)

In [ ]:
result_docs_pos = googlesearch_docs_df.groupby(['task_id', 'doc_position']).size().reset_index(name='count')
result_docs_pos 

In [ ]:
result_docs_page_viewed = googlesearch_docs_df.groupby(['task_id', 'doc_page_viewed']).size().reset_index(name='count')
result_docs_page_viewed

* Visualization

In [ ]:
def visualize_stackedbar(df, index): 
    pivot_df = df.pivot(index=index, columns='task_id', values='count').fillna(0)

    plt.figure(figsize=(6, 3))
    sns.set_palette("husl")
    pivot_df.plot(kind='bar', stacked=True)
    plt.xlabel(index)
    plt.ylabel('Count')
    plt.title(f'Count vs {index} for each Task')
    plt.legend(title='Task ID')
    plt.savefig(f"plot_output/count_{index}.jpg")

In [ ]:
visualize_stackedbar(result_docs_pos, 'doc_position')

In [ ]:
visualize_stackedbar(result_docs_page_viewed, 'doc_page_viewed')

# Inferential statistics

In [ ]:
new_posttasks_df = pd.merge(posttasks_df, tasks_df, on="task_id", how="inner")
new_posttasks_df = new_posttasks_df[['task_id', 'result-system', 'complexity-rate', 'satisfaction-rate', 'category' ]]
new_posttasks_df

In [ ]:
h2_df = new_posttasks_df[['task_id', 'result-system', 'category']]
h2_df